In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold
import lightgbm as lgb

In [ ]:
train = pd.read_csv("../input/lish-moa/train_features.csv")
test = pd.read_csv("../input/lish-moa/test_features.csv")
train_targets_scored = pd.read_csv("../input/lish-moa/train_targets_scored.csv")
train_targets_nonscored = pd.read_csv("../input/lish-moa/train_targets_nonscored.csv")
sub = pd.read_csv("../input/lish-moa/sample_submission.csv")

In [ ]:
def label_encoding(train: pd.DataFrame, test: pd.DataFrame, encode_cols):
    n_train = len(train)
    train = pd.concat([train, test], sort=False).reset_index(drop=True)
    for f in encode_cols:
        try:
            lbl = preprocessing.LabelEncoder()
            train[f] = lbl.fit_transform(list(train[f].values))
        except:
            print(f)
    test = train[n_train:].reset_index(drop=True)
    train = train[:n_train]
    return train, test

In [ ]:
def run_lgbm(target_col: str):
    
    X_train = train.drop(["sig_id"], axis=1)
    y_train = train_targets_scored[target_col]
    X_test = test.drop(["sig_id"], axis=1)

    y_preds = []
    models = []
    oof_train = np.zeros((len(X_train),))

    for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
        X_tr = X_train.loc[train_index, :]
        X_val = X_train.loc[valid_index, :]
        y_tr = y_train[train_index]
        y_val = y_train[valid_index]

        lgb_train = lgb.Dataset(X_tr,
                                y_tr,
                                categorical_feature=categorical_cols)

        lgb_eval = lgb.Dataset(X_val,
                               y_val,
                               reference=lgb_train,
                               categorical_feature=categorical_cols)

        model = lgb.train(params,
                          lgb_train,
                          valid_sets=[lgb_train, lgb_eval],
                          verbose_eval=10,
                          num_boost_round=1000,
                          early_stopping_rounds=10)


        oof_train[valid_index] = model.predict(X_val,
                                               num_iteration=model.best_iteration)
        y_pred = model.predict(X_test,
                               num_iteration=model.best_iteration)

        y_preds.append(y_pred)
        models.append(model)

    return oof_train, sum(y_preds) / len(y_preds)

# Preprocessing

We have to convert some categorical features into numbers in train and test. We can identify categorical features by `pd.DataFrame.select_dtypes`.

In [ ]:
train.head()

In [ ]:
train.select_dtypes(include=['object']).columns

In [ ]:
train, test = label_encoding(train, test, ['cp_type', 'cp_dose'])

In [ ]:
train.head()

# Modeling

In [ ]:
cv = KFold(n_splits=5, shuffle=True, random_state=0)

params = {
    'num_leaves': 24,
    'max_depth': 5,
    'objective': 'binary',
    'learning_rate': 0.01
}

categorical_cols = ['cp_type', 'cp_dose']
oof = train_targets_scored.copy()

In [ ]:
for target_col in train_targets_scored.columns:
    if target_col != "sig_id":
        _oof, _preds = run_lgbm(target_col)
        oof[target_col] = _oof
        sub[target_col] = _preds

In [ ]:
scores = []
for target_col in train_targets_scored.columns:
    if target_col != "sig_id":
        scores.append(log_loss(train_targets_scored[target_col], oof[target_col]))
print(np.mean(scores))

In [ ]:
sub.to_csv('submission.csv', index=False)